<a href="https://colab.research.google.com/github/MGonza20/Proyecto2_DataScience/blob/main/ExtentDamageModels_Proyect3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Leer la data del Proyecto 3

In [25]:
# Importar todas las librerias
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
!pip3 install tensorflow

In [49]:
# Librerias para implementar el modelo de RNN para clasificar
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [28]:
dataframe = pd.read_csv("./drive/Shareddrives/Data Science Proyecto 2/DataSets/train_clean.csv")
dataframe.head()

,filename,growth_stage,damage,extent,season
0,L398F01276C01S07896Rp26856.jpg,F,DR,0,LR2021
1,L355F02268C02S08916Rp30595.jpg,F,DR,0,LR2021
2,L134F00879C39S14248Rp42482.jpg,F,DR,10,SR2021
3,L1095F01359C01S08260Rp24587.jpg,F,DR,10,LR2021
4,L134F00777C39S14171Rp45993.jpg,F,DR,10,SR2021


In [29]:
dataframe['extent'] = dataframe['extent'].astype(int) / 10.0
dataframe.head()

,filename,growth_stage,damage,extent,season
0,L398F01276C01S07896Rp26856.jpg,F,DR,0.0,LR2021
1,L355F02268C02S08916Rp30595.jpg,F,DR,0.0,LR2021
2,L134F00879C39S14248Rp42482.jpg,F,DR,1.0,SR2021
3,L1095F01359C01S08260Rp24587.jpg,F,DR,1.0,LR2021
4,L134F00777C39S14171Rp45993.jpg,F,DR,1.0,SR2021


# Implement RNN Model Clasification

In [34]:
# Data Processing
label_encoder = LabelEncoder()
dataframe['growth_stage_encoded'] = label_encoder.fit_transform(dataframe['growth_stage'])
dataframe['damage_encoded'] = label_encoder.fit_transform(dataframe['damage'])

# Divide the data in the sets of training and test
X = dataframe[['growth_stage_encoded', 'damage_encoded']]
y = dataframe['extent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
# Create the RNN model with Keras
model = Sequential()
model.add(Embedding(input_dim=len(label_encoder.classes_), output_dim=10, input_length=2))
model.add(LSTM(50))
model.add(Dense(units=1, activation='sigmoid'))

In [36]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
652/652 [==============================] - 8s 8ms/step - loss: -19.4193 - accuracy: 0.2843 - val_loss: -41.0507 - val_accuracy: 0.4604
Epoch 2/50
652/652 [==============================] - 3s 4ms/step - loss: -64.0931 - accuracy: 0.6298 - val_loss: -78.2056 - val_accuracy: 0.6466
Epoch 3/50
652/652 [==============================] - 3s 4ms/step - loss: -98.7924 - accuracy: 0.6335 - val_loss: -109.0787 - val_accuracy: 0.6470
Epoch 4/50
652/652 [==============================] - 3s 4ms/step - loss: -130.5342 - accuracy: 0.6338 - val_loss: -138.8845 - val_accuracy: 0.6470
Epoch 5/50
652/652 [==============================] - 4s 6ms/step - loss: -161.5143 - accuracy: 0.6339 - val_loss: -168.0396 - val_accuracy: 0.6470
Epoch 6/50
652/652 [==============================] - 4s 6ms/step - loss: -192.0674 - accuracy: 0.6338 - val_loss: -196.9718 - val_accuracy: 0.6470
Epoch 7/50
652/652 [==============================] - 4s 6ms/step - loss: -222.3795 - accuracy: 0.6343 - val_loss: -2

In [38]:
# Check the model
accuracy = model.evaluate(X_test, y_test)[1]
print(f'Accuracy: {accuracy}')

163/163 [==============================] - 1s 3ms/step - loss: -1456.9596 - accuracy: 0.6470
Accuracy: 0.6469798684120178


Para este primer intento se obtuvo Accuracy de 0.6469798684120178.

Acontinación vamos a probar a cambiar algunos parametros para ver si podemos mejorar los resultados de clasificación.

In [47]:
label_encoder = LabelEncoder()
dataframe['growth_stage_encoded'] = label_encoder.fit_transform(dataframe['growth_stage'])
dataframe['damage_encoded'] = label_encoder.fit_transform(dataframe['damage'])

X = dataframe[['growth_stage_encoded', 'damage_encoded']]
y = dataframe['extent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(input_dim=len(label_encoder.classes_), output_dim=10, input_length=2))
model.add(LSTM(128, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='linear'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
652/652 [==============================] - 22s 17ms/step - loss: 2.2810 - mae: 0.9793 - val_loss: 2.0353 - val_mae: 0.9008
Epoch 2/20
652/652 [==============================] - 8s 13ms/step - loss: 1.9955 - mae: 0.9106 - val_loss: 1.8523 - val_mae: 0.8455
Epoch 3/20
652/652 [==============================] - 7s 11ms/step - loss: 1.9842 - mae: 0.8969 - val_loss: 1.8307 - val_mae: 0.8055
Epoch 4/20
652/652 [==============================] - 9s 13ms/step - loss: 1.9804 - mae: 0.8948 - val_loss: 1.8753 - val_mae: 0.8699
Epoch 5/20
652/652 [==============================] - 9s 13ms/step - loss: 1.9970 - mae: 0.8989 - val_loss: 1.8031 - val_mae: 0.7790
Epoch 6/20
652/652 [==============================] - 8s 13ms/step - loss: 1.9932 - mae: 0.8987 - val_loss: 1.8326 - val_mae: 0.8528
Epoch 7/20
652/652 [==============================] - 9s 14ms/step - loss: 1.9682 - mae: 0.8856 - val_loss: 1.8034 - val_mae: 0.7961
Epoch 8/20
652/652 [==============================] - 7s 10ms/step -

In [48]:
accuracy = model.evaluate(X_test, y_test)[1]
print(f'Accuracy: {accuracy}')

163/163 [==============================] - 1s 4ms/step - loss: 1.8233 - mae: 0.8361
Accuracy: 0.8360713720321655


 Se pudo obtener un mejor Accuracy de 0.8360713720321655 para este modelo con nuevos ajutes.